In [1]:
import torch
from train import train_loop
from data_module import EEGDataModule
from model import ViTransformer, LSTM, ConvLSTM

/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device(0)

In [4]:
import numpy as np
def transform(dataset):
    N, F, L = dataset.shape
    return dataset.reshape(N, F * 5, L // 5)

In [5]:
train_params = {
    "train_epochs": 10,
    "accumulate_grad_batches": 1,
    "test_size": 0.2,
    "random_state": 42,
    "data_dir": "../../Data/",
    "train_batch_size": 64,
    "eval_batch_size": 1
}

In [6]:
dataset = EEGDataModule(args=train_params)

In [7]:
dataset.setup()

INFO:data_module:Training data shape: (1692, 22, 1000)
INFO:data_module:Training labels shape: (1692,)


In [8]:
train, valid = dataset.train_dataloader(), dataset.val_dataloader()

INFO:data_module:loaded 1692 train data instances
INFO:data_module:loaded 423 train data instances


In [9]:
model = ConvLSTM()

In [10]:
from torchinfo import summary
# Only uses outputs of modules.
print(summary(ConvLSTM()))

Layer (type:depth-idx)                   Param #
ConvLSTM                                 --
├─Conv2d: 1-1                            390
├─Conv2d: 1-2                            4,965
├─AvgPool2d: 1-3                         --
├─LSTM: 1-4                              4,224
├─Linear: 1-5                            132
├─ELU: 1-6                               --
Total params: 9,711
Trainable params: 9,711
Non-trainable params: 0


In [11]:
loss_hist, acc_hist, val_loss_hist, val_acc_hist = train_loop(model, train, valid, device)

100%|██████████████████████████████| 100/100 [00:47<00:00,  2.09it/s, acc=0.924, val_acc=0.262]
